## Country Club Case Study

In [1]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

### Questions:

Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

In [2]:
query_1= "SELECT name FROM Facilities WHERE membercost >0;"

In [3]:
with engine.connect() as conn:
    q1_df = pd.read_sql_query(query_1, conn)
    
print(q1_df)

             name
0  Tennis Court 1
1  Tennis Court 2
2  Massage Room 1
3  Massage Room 2
4    Squash Court


Q2: How many facilities do not charge a fee to members? 

In [5]:
query_2 = "SELECT COUNT(name) FROM Facilities WHERE membercost =0;"

In [6]:
with engine.connect() as conn:
    q2_df = pd.read_sql_query(query_2, conn)
    
print(q2_df)

   COUNT(name)
0            4


Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. 

In [7]:
query_3 = "SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE membercost > 0 AND membercost < (0.2 * monthlymaintenance)"

In [8]:
with engine.connect() as conn:
    q3_df = pd.read_sql_query(query_3, conn)
    
print(q3_df)

   facid            name  membercost  monthlymaintenance
0      0  Tennis Court 1         5.0                 200
1      1  Tennis Court 2         5.0                 200
2      4  Massage Room 1         9.9                3000
3      5  Massage Room 2         9.9                3000
4      6    Squash Court         3.5                  80


Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. 

In [9]:
query_4 = "SELECT * FROM Facilities WHERE facid IN (1,5)"

In [10]:
with engine.connect() as conn:
    q4_df = pd.read_sql_query(query_4, conn)
    
print(q4_df)

   facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0         25           8000   
1      5  Massage Room 2         9.9         80           4000   

   monthlymaintenance  
0                 200  
1                3000  


Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [11]:
query_5 = "SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance < 100 THEN 'cheap'ELSE 'expensive' END AS cost FROM Facilities"

In [12]:
with engine.connect() as conn:
    q5_df = pd.read_sql_query(query_5, conn)
    
print(q5_df)

              name  monthlymaintenance       cost
0   Tennis Court 1                 200  expensive
1   Tennis Court 2                 200  expensive
2  Badminton Court                  50      cheap
3     Table Tennis                  10      cheap
4   Massage Room 1                3000  expensive
5   Massage Room 2                3000  expensive
6     Squash Court                  80      cheap
7    Snooker Table                  15      cheap
8       Pool Table                  15      cheap


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [13]:
query_6 = "SELECT firstname, surname, joindate FROM Members WHERE joindate = (SELECT MAX(joindate) FROM Members)"

In [14]:
with engine.connect() as conn:
    q6_df = pd.read_sql_query(query_6, conn)
    
print(q6_df)

  firstname surname             joindate
0    Darren   Smith  2012-09-26 18:08:45


Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. 

In [19]:
query_7= "SELECT firstname || ' ' || surname AS Fullname, Facility_Name FROM Members LEFT JOIN (SELECT DISTINCT facid, memid, f.name AS Facility_Name FROM Bookings INNER JOIN Facilities AS f USING (facid) WHERE facid IN (0,1) ORDER BY facid, memid) AS fac_book USING (memid) WHERE Facility_Name IS NOT NULL ORDER BY surname;"

In [20]:
with engine.connect() as conn:
    q7_df = pd.read_sql_query(query_7, conn)
    
print(q7_df)

             Fullname   Facility_Name
0      Florence Bader  Tennis Court 1
1      Florence Bader  Tennis Court 2
2          Anne Baker  Tennis Court 1
3       Timothy Baker  Tennis Court 1
4          Anne Baker  Tennis Court 2
5       Timothy Baker  Tennis Court 2
6          Tim Boothe  Tennis Court 1
7          Tim Boothe  Tennis Court 2
8      Gerald Butters  Tennis Court 1
9      Gerald Butters  Tennis Court 2
10        Joan Coplin  Tennis Court 1
11      Erica Crumpet  Tennis Court 1
12         Nancy Dare  Tennis Court 1
13         Nancy Dare  Tennis Court 2
14     Jemima Farrell  Tennis Court 1
15      David Farrell  Tennis Court 1
16     Jemima Farrell  Tennis Court 2
17      David Farrell  Tennis Court 2
18        GUEST GUEST  Tennis Court 1
19        GUEST GUEST  Tennis Court 2
20    Matthew Genting  Tennis Court 1
21          John Hunt  Tennis Court 1
22          John Hunt  Tennis Court 2
23        David Jones  Tennis Court 1
24      Douglas Jones  Tennis Court 1
25        Da

Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [85]:
query_8 = "SELECT firstname || ' ' || surname AS Fullname, f.name AS Facility, CASE WHEN memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END AS Cost FROM Bookings AS b INNER JOIN Members AS m USING(memid) INNER JOIN Facilities AS f USING(facid) WHERE DATE(starttime) = '2012-09-14' AND CASE WHEN memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END > 30 ORDER BY Cost DESC;"

In [86]:
with engine.connect() as conn:
    q8_df = pd.read_sql_query(query_8, conn)
    
print(q8_df)

          Fullname        Facility   Cost
0      GUEST GUEST  Massage Room 2  320.0
1      GUEST GUEST  Massage Room 1  160.0
2      GUEST GUEST  Massage Room 1  160.0
3      GUEST GUEST  Massage Room 1  160.0
4      GUEST GUEST  Tennis Court 2  150.0
5      GUEST GUEST  Tennis Court 1   75.0
6      GUEST GUEST  Tennis Court 1   75.0
7      GUEST GUEST  Tennis Court 2   75.0
8      GUEST GUEST    Squash Court   70.0
9   Jemima Farrell  Massage Room 1   39.6
10     GUEST GUEST    Squash Court   35.0
11     GUEST GUEST    Squash Court   35.0


Q9: This time, produce the same result as in Q8, but using a subquery.

In [92]:
query_9 = "SELECT * FROM (SELECT firstname || ' ' || surname AS Fullname, f.name AS Facility, CASE WHEN memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END AS Cost FROM Bookings AS b INNER JOIN Members AS m USING(memid) INNER JOIN Facilities AS f USING(facid) WHERE DATE(starttime) = '2012-09-14' ORDER BY Cost DESC) AS subquery WHERE Cost > 30;"

In [93]:
with engine.connect() as conn:
    q9_df = pd.read_sql_query(query_9, conn)
    
print(q9_df)

          Fullname        Facility   Cost
0      GUEST GUEST  Massage Room 2  320.0
1      GUEST GUEST  Massage Room 1  160.0
2      GUEST GUEST  Massage Room 1  160.0
3      GUEST GUEST  Massage Room 1  160.0
4      GUEST GUEST  Tennis Court 2  150.0
5      GUEST GUEST  Tennis Court 1   75.0
6      GUEST GUEST  Tennis Court 1   75.0
7      GUEST GUEST  Tennis Court 2   75.0
8      GUEST GUEST    Squash Court   70.0
9   Jemima Farrell  Massage Room 1   39.6
10     GUEST GUEST    Squash Court   35.0
11     GUEST GUEST    Squash Court   35.0


### PART 2: SQLite

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [232]:
query_10 = """
SELECT Facility, SUM(Cost) AS Total_Revenue
FROM (SELECT 
f.name AS Facility,
CASE WHEN memid = 0 
THEN f.guestcost * b.slots 
ELSE f.membercost * b.slots END AS Cost 
FROM Bookings AS b
INNER JOIN Members AS m 
USING(memid) 
INNER JOIN Facilities  AS f 
USING(facid)) AS subquery
GROUP BY Facility
ORDER BY Total_Revenue;
"""


In [233]:
with engine.connect() as conn:
    q10_df = pd.read_sql_query(query_10, conn)
    
print(q10_df)

          Facility  Total_Revenue
0     Table Tennis          180.0
1    Snooker Table          240.0
2       Pool Table          270.0
3  Badminton Court         1906.5
4     Squash Court        13468.0
5   Tennis Court 1        13860.0
6   Tennis Court 2        14310.0
7   Massage Room 2        14454.6
8   Massage Room 1        50351.6


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [178]:
query_11 = """
SELECT m1.firstname Membername, m1.surname AS Member_surname, m2.firstname || ' ' || m2.surname AS RecommendedBy
FROM Members AS m1
INNER JOIN Members AS m2
ON m1.recommendedby = m2.memid
ORDER BY Member_surname, Membername
"""

In [179]:
with engine.connect() as conn:
    q11_df = pd.read_sql_query(query_11, conn)
    
print(q11_df)

   Membername     Member_surname      RecommendedBy
0    Florence              Bader    Ponder Stibbons
1        Anne              Baker    Ponder Stibbons
2     Timothy              Baker     Jemima Farrell
3         Tim             Boothe         Tim Rownam
4      Gerald            Butters       Darren Smith
5        Joan             Coplin      Timothy Baker
6       Erica            Crumpet        Tracy Smith
7       Nancy               Dare    Janice Joplette
8     Matthew            Genting     Gerald Butters
9        John               Hunt  Millicent Purview
10      David              Jones    Janice Joplette
11    Douglas              Jones        David Jones
12     Janice           Joplette       Darren Smith
13       Anna          Mackenzie       Darren Smith
14    Charles               Owen       Darren Smith
15      David             Pinker     Jemima Farrell
16  Millicent            Purview        Tracy Smith
17  Henrietta             Rumney    Matthew Genting
18  Ramnares

Q12: Find the facilities with their usage by member, but not guests

In [192]:
query_12 = """
SELECT f.name AS Facility, COUNT(DISTINCT b.memid) AS mems_have_used
FROM Facilities as f
INNER JOIN Bookings AS b 
USING(facid)
GROUP BY Facility;
"""

In [193]:
with engine.connect() as conn:
    q12_df = pd.read_sql_query(query_12, conn)
    
print(q12_df)

          Facility  mems_have_used
0  Badminton Court              25
1   Massage Room 1              25
2   Massage Room 2              13
3       Pool Table              28
4    Snooker Table              23
5     Squash Court              25
6     Table Tennis              26
7   Tennis Court 1              24
8   Tennis Court 2              22


Q13: Find the facilities usage by month, but not guests 

In [240]:
query_13 =  """ 

SELECT Facility, Month, SUM(Use) AS Monthly_Usage
FROM (SELECT f.name AS Facility, strftime('%m', DATE(starttime)) as Month, 
CASE WHEN memid = 0 THEN 0
ELSE 1 END AS Use
FROM Facilities as f
INNER JOIN Bookings AS b 
USING(facid)) AS subquery
GROUP BY Facility, Month

    """ 

In [242]:
with engine.connect() as conn:
    q13_df = pd.read_sql_query(query_13, conn)
    
print(q13_df)

           Facility Month  SUM(Use)
0   Badminton Court    07        51
1   Badminton Court    08       132
2   Badminton Court    09       161
3    Massage Room 1    07        77
4    Massage Room 1    08       153
5    Massage Room 1    09       191
6    Massage Room 2    07         4
7    Massage Room 2    08         9
8    Massage Room 2    09        14
9        Pool Table    07       103
10       Pool Table    08       272
11       Pool Table    09       408
12    Snooker Table    07        68
13    Snooker Table    08       154
14    Snooker Table    09       199
15     Squash Court    07        23
16     Squash Court    08        85
17     Squash Court    09        87
18     Table Tennis    07        48
19     Table Tennis    08       143
20     Table Tennis    09       194
21   Tennis Court 1    07        65
22   Tennis Court 1    08       111
23   Tennis Court 1    09       132
24   Tennis Court 2    07        41
25   Tennis Court 2    08       109
26   Tennis Court 2    09   